In [ ]:
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms
import torchvision.models as models
import torchinfo
import matplotlib.pyplot as plt
import torch.nn.functional as F
from tqdm import tqdm

import numpy as np
import csv

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 64x64
    transforms.ToTensor(),  # Convert images to tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the images
])

train_dir = '/kaggle/input/iith-dl-contest-2024/train/train'
train_data  = datasets.ImageFolder(train_dir, transform = transform)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
len(train_loader)

In [ ]:
resnetmodel = models.resnet18(weights = None)

In [ ]:
numftrs = resnetmodel.fc.in_features
resnetmodel.fc = nn.Linear(numftrs, 50)

for param in resnetmodel.fc.parameters():
    param.requires_grad = True

In [ ]:
torchinfo.summary(resnetmodel, input_size = (32, 3, 224, 224))

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(resnetmodel.parameters(), lr = 0.001)

total_steps = len(train_loader)

for epoch in range(20):

    Loss = 0
    correct = 0
    total = 0
    for i , (images, label) in tqdm(enumerate(train_loader)):

        images = images.to(device)
        label = label.to(device)

        outputs =  resnetmodel(images)
        loss = criterion(outputs, label)
        
        with torch.no_grad():
            Loss += loss*images.shape[0]
            _, predicted = torch.max(outputs, 1)
            correct += torch.sum(predicted == label).item()
            total += label.shape[0]

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print("epoch ", epoch, " : Loss = ", Loss.item()/len(train_data), "accuracy = ", correct/total)
    
    if((epoch+1)>15):
        str = "resnetmodelweights{}.pth".format(epoch+1)
        torch.save(resnetmodel.state_dict(), str)

In [ ]:
classes = train_data.class_to_idx
idx_to_class = {idx: class_name for class_name, idx in classes.items()}

test_dir = '/kaggle/input/iith-dl-contest-2024/test'
test_data  = datasets.ImageFolder(test_dir, transform = transform)

test_loader = torch.utils.data.DataLoader(test_data, batch_size = 32, shuffle = False)

In [ ]:
list_of_outs = []

for i, (images, labels) in tqdm(enumerate(test_loader)):
    
    images = images.to(device)
    
    outputs = resnetmodel(images)
    _, outputs = torch.max(outputs.data, 1)

    list_of_outs.append(outputs)
        
outs = torch.cat(list_of_outs)

In [ ]:
right = outs.cpu().numpy()
right = np.array([idx_to_class[_] for _ in right], dtype = object)
right.shape

left = []
for i in range(38366):
    s = "{}.JPEG".format(i)
    left.append(s)
left = np.sort(left)

table = np.transpose(np.array([left, right], dtype = object))

with open('submission_resnet18.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['ID', 'Category'])
    writer.writerows(table)